# FFmpeg + Python in Google Colab (v5)
**Polished build** — safe accelerator detection, robust concat section, and no `%%bash` magics.

## 1) Install FFmpeg + helpers

In [ ]:
#@title Install FFmpeg + Python packages { display-mode: "form" }
!apt-get -y update -qq
!apt-get -y install -qq ffmpeg
!pip -q install ffmpeg-python imageio imageio-ffmpeg opencv-python rich

## 2) Check your runtime (safe GPU/TPU/CPU detection)

In [ ]:
# Detect accelerator safely (GPU/TPU/CPU)
import shutil, subprocess, os

def print_block(s): 
    print("\n" + "="*30 + "\n" + s + "\n" + "="*30)

# GPU (NVIDIA) — only run nvidia-smi if present
if shutil.which("nvidia-smi"):
    out = subprocess.run(["nvidia-smi"], stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    print_block("NVIDIA GPU detected")
    print(out.stdout)
else:
    print_block("No NVIDIA GPU visible (CPU or TPU). That's fine for software encoding.")

# TPU (Colab has an env var when TPU is present)
if "COLAB_TPU_ADDR" in os.environ:
    print_block(f"TPU detected at {os.environ['COLAB_TPU_ADDR']}")

## 3) (Optional) Mount Google Drive

In [ ]:
from google.colab import drive  # type: ignore
drive.mount('/content/drive')

## 4) Create a sample clip to experiment with

In [ ]:
# 5s 720p test pattern + tone
!ffmpeg -hide_banner -y   -f lavfi -i testsrc=size=1280x720:rate=30   -f lavfi -i sine=frequency=440:sample_rate=48000:duration=5   -shortest -c:v libx264 -pix_fmt yuv420p -preset veryfast -crf 23   -c:a aac -b:a 128k sample.mp4
!ls -lh sample.mp4

## 5) Inspect with ffprobe

In [ ]:
!ffprobe -v error -print_format json -show_format -show_streams sample.mp4 | python -m json.tool

## 6) Common edits

In [ ]:
# Trim (copy) — keyframe aligned
!ffmpeg -hide_banner -y -ss 1 -to 4 -i sample.mp4 -c copy sample_trim_copy.mp4
!ffprobe -v error -show_entries format=duration -of default=nw=1:nk=1 sample_trim_copy.mp4

# Exact trim (re-encode)
!ffmpeg -hide_banner -y -ss 1 -to 4 -i sample.mp4 -c:v libx264 -preset veryfast -crf 20 -c:a aac -b:a 128k exact_trim.mp4
!ffprobe -v error -show_entries format=duration -of default=nw=1:nk=1 exact_trim.mp4

In [ ]:
# Resize + fps
!ffmpeg -hide_banner -y -i sample.mp4 -vf "scale=854:-2,fps=24" -c:v libx264 -preset veryfast -crf 23 -c:a aac -b:a 128k sample_854x480_24fps.mp4

### Concatenate clips (robust method)

In [ ]:
# Create two short segments
!ffmpeg -hide_banner -y -i sample.mp4 -t 2 -c copy part1.mp4
!ffmpeg -hide_banner -y -ss 2 -i sample.mp4 -t 2 -c copy part2.mp4

# Write concat list file correctly from Python (quotes recommended)
with open("concat.txt", "w") as f:
    f.write("file 'part1.mp4'\n")
    f.write("file 'part2.mp4'\n")

# Concat using demuxer
!ffmpeg -hide_banner -y -f concat -safe 0 -i concat.txt -c copy sample_concat.mp4
!ls -lh sample_concat.mp4

### Side-by-side (hstack) + overlay PiP

In [ ]:
!ffmpeg -hide_banner -y -i sample.mp4 -i sample.mp4   -filter_complex "[0:v]scale=640:-2[left];[1:v]scale=640:-2[right];[left][right]hstack=inputs=2[sbs];[sbs][1:v]overlay=W-w-20:H-h-20:enable='between(t,1,4)'"   -map "[sbs]" -map 0:a -c:v libx264 -crf 22 -preset veryfast -c:a aac -b:a 128k sxs_overlay.mp4

### Subtitles (burn-in)

In [ ]:
# Write an SRT file from Python, then burn in
srt = '''1
00:00:00,000 --> 00:00:02,000
Hello from FFmpeg!

2
00:00:02,000 --> 00:00:05,000
These are hardcoded subtitles.
'''
with open('demo.srt','w') as f:
    f.write(srt)

!ffmpeg -hide_banner -y -i sample.mp4 -vf subtitles=demo.srt -c:v libx264 -crf 22 -preset veryfast -c:a copy subbed.mp4

### GIF (palette method)

In [ ]:
!ffmpeg -hide_banner -y -i sample.mp4 -vf "fps=12,scale=480:-2,split[s0][s1];[s0]palettegen[p];[s1][p]paletteuse=dither=bayer" sample.gif
!ls -lh sample.gif

## 7) Bitrate & 2-pass

In [ ]:
# CRF encode
!ffmpeg -hide_banner -y -i sample.mp4 -c:v libx264 -crf 20 -preset slow -c:a aac -b:a 128k crf20_slow.mp4

# Two-pass (target 2.5 Mbps)
!ffmpeg -hide_banner -y -i sample.mp4 -c:v libx264 -b:v 2500k -pass 1 -an -f mp4 /dev/null
!ffmpeg -hide_banner -y -i sample.mp4 -c:v libx264 -b:v 2500k -pass 2 -c:a aac -b:a 128k two_pass_2_5Mbps.mp4
!rm -f ffmpeg2pass-0.log ffmpeg2pass-0.log.mbtree

## 8) Filters (scale, crop, pad, EQ)

In [ ]:
!ffmpeg -hide_banner -y -i sample.mp4   -vf "crop=in_w:in_h*0.8:0:in_h*0.1,scale=960:-2,pad=1280:720:(1280-iw)/2:(720-ih)/2:black,eq=contrast=1.1:brightness=0.03:saturation=1.05"   -c:v libx264 -crf 21 -preset veryfast -c:a aac -b:a 128k styled.mp4

## 9) Audio tasks

In [ ]:
# Loudness normalize
!ffmpeg -hide_banner -y -i sample.mp4 -af loudnorm=I=-16:TP=-1.5:LRA=11 -c:v copy -c:a aac -b:a 192k loudnorm.mp4

# Sidechain ducking (VO over music example)
!ffmpeg -hide_banner -y -i sample.mp4 -f lavfi -i sine=frequency=220:sample_rate=48000:duration=5   -filter_complex "[0:a][1:a]sidechaincompress=threshold=0.05:ratio=8:attack=20:release=250[ducked]"   -map 0:v -map "[ducked]" -c:v copy -c:a aac -b:a 128k ducked_music.mp4

## 10) Pythonic pipelines (`ffmpeg-python`)

In [ ]:
import ffmpeg, subprocess, json
probe = ffmpeg.probe('sample.mp4')
video_streams = [s for s in probe['streams'] if s.get('codec_type')=='video']
print('Video codec:', video_streams[0].get('codec_name'))
(ffmpeg
  .input('sample.mp4')
  .filter('scale', 960, -2)
  .output('py_transcoded.mp4', vcodec='libx264', crf=23, preset='veryfast', acodec='aac', **{'b:a':'128k'})
  .overwrite_output()
  .run()
)
subprocess.run(['ffprobe','-v','error','-show_entries','format=duration','-of','default=nw=1:nk=1','py_transcoded.mp4'])

## 11) OpenCV read/write frames

In [ ]:
import cv2
cap = cv2.VideoCapture('sample.mp4')
w,h = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # portable fallback
out = cv2.VideoWriter('cv2_out.mp4', fourcc, fps, (w,h))
while True:
    ok, frame = cap.read()
    if not ok: break
    out.write(frame)
cap.release(); out.release(); print('Wrote cv2_out.mp4')

## 12) Cheatsheet: useful knobs

- H.264 (libx264): `-crf 18–28`, `-preset ultrafast..placebo`, `-tune film|animation|grain`
- VP9 (libvpx-vp9): `-b:v 0 -crf 30–40`, `-row-mt 1`, `-tile-columns 2`
- AV1 (libaom-av1): `-crf 28–40`, `-cpu-used 4–8`
- Audio: AAC `-c:a aac -b:a 128k`, Opus `-c:a libopus -b:a 96k`
- Filters: `scale`, `fps`, `crop`, `pad`, `eq`, `overlay`, `hstack/vstack`
- Mux: `-movflags +faststart`, explicit `-map` for streams